In [1]:
%pip  install --upgrade transformers optimum

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
# %pip install -U wheel

In [4]:
# %pip install --upgrade auto-gptq

In [5]:
# %pip install -v flash-attn==2.1.1 --no-build-isolation

In [6]:
import torch
print(torch.__version__)
print (torch.cuda.is_available())

2.1.2+cu121
True


In [7]:
%pip install accelerate

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install --upgrade bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install --upgrade scipy

Note: you may need to restart the kernel to use updated packages.


In [10]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"


In [11]:
%pip install ipywidgets
# from .autonotebook import tqdm as notebook_tqdm

Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install --upgrade setuptools

Note: you may need to restart the kernel to use updated packages.


In [13]:
from transformers import BitsAndBytesConfig
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
)

In [14]:
from transformers import AutoModelForCausalLM, pipeline

model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}
)
# # model_name_or_path = "TheBloke/dolphin-2.7-mixtral-8x7b-GPTQ"
# model_name_or_path = "mistralai/Mixtral-8x7B-Instruct-v0.1"
# # To use a different branch, change revision
# # For example: revision="gptq-4bit-128g-actorder_True"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                                device_map="auto",
#                                                torch_dtype=torch.float16,
#                                                offload_folder="offload",
#                                                trust_remote_code=True,
#                                                low_cpu_mem_usage=True,
#                                                quantization_config=bnb_config)
                                             

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)



Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [21]:
print(model)


MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear4bit(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBLockSparseTop2MLP(
              (w1): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (w2): Linear4bit(in_features=14336, out_features=4096, bias=False)
              (w3): Linear4bit(in_features=4096, out_feat

In [16]:
from transformers import  AutoTokenizer

#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left'

In [17]:
prompt="Summarize your thoughts on AI hallucinations"

In [18]:
input_ids = tokenizer.encode(prompt, return_tensors='pt',padding=True).cuda()
output = model.generate(inputs=input_ids, pad_token_id=tokenizer.unk_token_id,do_sample=True,temperature=0.4, repetition_penalty=1.1, top_p=0.95, top_k=40, max_new_tokens=512).cuda()
print(tokenizer.decode(output[0]))

<s> Summarize your thoughts on AI hallucinations</s>ef{sec:hallucination}

AI hallucinations are a significant challenge in the development and deployment of large language models like me. They refer to situations where the model generates text that is coherent and fluent but factually incorrect or nonsensical. This can occur due to various reasons, such as the model's training data containing errors or contradictions, or the model's inability to distinguish between real and fake information.

Hallucinations can have serious consequences, especially in high-stakes applications such as healthcare, finance, or legal contexts. Therefore, it is crucial to address this issue and find ways to mitigate it. One approach is to improve the quality and diversity of the training data, ensuring that it is as accurate and consistent as possible. Another approach is to develop post-hoc evaluation methods that can detect and flag instances of hallucination in the model's outputs. Additionally, incorpo

In [19]:
print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.unk_token_id,
    do_sample=True,
    temperature=0.4,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt)[0]['generated_text'])

*** Pipeline:
Summarize your thoughts on AI hallucinations and the importance of building trust with users.

AI hallucinations are a significant issue that can lead to mistrust and misinformation. They occur when an AI model generates outputs that are factually incorrect or nonsensical. This problem is particularly prevalent in language models, where the model might generate plausible-sounding but entirely made-up information.

To build trust with users, it's crucial to acknowledge this issue and be transparent about the limitations of AI technology. Developers should implement strategies to detect and mitigate hallucinations, such as using multiple models to cross-check results or incorporating fact-checking mechanisms. Additionally, educating users about the capabilities and limitations of AI can help manage expectations and foster a more realistic understanding of the technology. By being proactive in addressing AI hallucinations and promoting transparency, we can work towards build

In [20]:
messages = [
    {"role": "user", "content": "Tell me a recipe for Upma."},
]

print("\n\n*** Prompt:")
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    return_tensors="pt",
)
print(tokenizer.decode(input_ids[0]))

print("\n\n*** Generate:")
with torch.autocast("cuda", dtype=torch.bfloat16):
    output = model.generate(
        input_ids=input_ids.to("cuda"),
        max_new_tokens=1024,
        return_dict_in_generate=True,
    )

response = tokenizer.decode(
    output["sequences"][0][len(input_ids[0]):], 
    skip_special_tokens=True
)
print(response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




*** Prompt:
<s> [INST] Tell me a recipe for Upma. [/INST]


*** Generate:
Sure, here's a simple recipe for Upma, a popular Indian breakfast dish made with semolina:

Ingredients:

* 1 cup semolina (rava)
* 2 tablespoons oil or ghee
* 1 teaspoon mustard seeds
* 1 teaspoon cumin seeds
* 1 onion, finely chopped
* 2 green chilies, finely chopped
* 1 teaspoon ginger, grated or minced
* 1 teaspoon curry leaves (optional)
* 1 cup mixed vegetables (such as peas, carrots, beans, and corn)
* 2 cups water
* 1 teaspoon salt (or to taste)
* 1/2 teaspoon turmeric powder
* 1/2 teaspoon coriander powder
* 1/4 teaspoon cumin powder
* 1/4 teaspoon garam masala
* 2 tablespoons lemon juice
* 2 tablespoons chopped cilantro

Instructions:

1. Dry roast the semolina in a pan over medium heat until it turns light brown and gives off a nutty aroma. Transfer to a plate and set aside.
2. Heat oil or ghee in the same pan over medium heat. Add mustard seeds and cumin seeds and let them crackle.
3. Add onion, gre